In [1]:
from scipy.spatial import cKDTree
import pyfits, mechanize, bz2, os
from matplotlib import pyplot as pl
from numpy import *
from scipy.optimize import leastsq
import astropy.stats as sta
from scipy import ndimage
from glob import glob
import pandas as pd  #為了讀csv檔。csv檔是以逗點分割的檔案。
import requests
from bs4 import BeautifulSoup
from astropy.io import ascii
import concurrent.futures
from time import sleep
from astropy.table import Table

In [3]:
header = {
'Upgrade-Insecure-Requests': '1',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
'Referer': 'http://irsa.ipac.caltech.edu/ibe/docs/ptf/images/level1/',
'Cookie': 'Cookie:"__utma=253438417.1413587276.1479455055.1486991316.1487038069.11; __utmz=253438417.1479459352.2.2.utmcsr=yahoo|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); usrkey="7820253d-5a48-472a-b8a4-44fa98bcc713/ncuiarex@astro.ncu.edu.tw"; __utmc=253438417; JOSSO_SESSIONID=3CBC377366AB0183013199AB98CACC73'
}    

In [18]:
os.chdir('D:\kepler_ptf_research\k2_c1')

In [6]:
dfs = pd.read_csv('K2_M_C1_ptf_h.csv')

In [47]:
def input_ra_dec(ra, dec, epic):
    
    #df = pd.read_csv('ptf_ha_substraction_k2c1.csv')
    #column_names = ['EPIC','HA663','HA656','HA663-HA656']  
    
    #if epic not in list(df['EPIC'])  :
    
    res = requests.get("http://irsa.ipac.caltech.edu/ibe/search/ptf/images/level1?POS={0},{1}".format(ra, dec), headers = header) 
    #print (res.text) #印出網頁內容
    soup = BeautifulSoup(res.text, 'lxml')
    data = ascii.read(soup.text)
    ha656_list = []
    ha663_list = []
    a = data['filter']
    af3 = data['afilename3']
    for i in range(0, len(af3)):
        if a[i] == "HA656":
             ha656_list.append(af3[i])
        if a[i] == "HA663":
             ha663_list.append(af3[i])


    url_ha663 = "http://irsa.ipac.caltech.edu/ibe/data/ptf/images/level1/"+ha663_list[0]
    url_ha656 = "http://irsa.ipac.caltech.edu/ibe/data/ptf/images/level1/"+ha656_list[0]
    ha663_filename = url_ha663.split('/')[-1]
    ha656_filename = url_ha656.split('/')[-1]
    # NOTE the stream=True parameter


    os.chdir("D:/kepler_ptf_research/k2_c1/python_PTF_data_k2c1")  #for windows
    #os.chdir("/media/careyjohn/DATA/python_PTF_data_k2c1")  #for Linux

    if not os.path.exists('ktwo'+str(epic)+'_C1'): # 若無該資料夾，則新建
        os.mkdir('ktwo'+str(epic)+'_C1')

    #os.mkdir('ktwo'+str(epic)+'_C1')

    os.chdir("D:/kepler_ptf_research/k2_c1/python_PTF_data_k2c1/ktwo"+str(epic)+"_C1")  #for windows
    #os.chdir("/media/careyjohn/DATA/python_PTF_data_k2c1/ktwo"+str(epic)+"_C1") # for Linux


    def P_ha656(str1):

        for i in range(len(str1)):
            if str1[i] == 'P':
                return str1[i:]

    def P_ha663(str2):

        for i in range(len(str2)):
            if str2[i] == 'P':
                return str2[i:]

    if not os.path.exists(P_ha663(ha663_list[0])) :

        r_ha663 = requests.get( url_ha663, stream=True, headers = header)
        with open(ha663_filename, 'wb') as f663:
            for chunk in r_ha663.iter_content(chunk_size=1024): 
                if chunk: # filter out keep-alive new chunks
                    f663.write(chunk)

    if not os.path.exists(P_ha656(ha656_list[0])):

        r_ha656 = requests.get( url_ha656, stream=True, headers = header)
        with open(ha656_filename, 'wb') as f656:
            for chunk in r_ha656.iter_content(chunk_size=1024): 
                if chunk: # filter out keep-alive new chunks
                    f656.write(chunk)
        #return ha663_filename, ha656_filename

    ID = 'ktwo'+str(epic)+'_C1'  
    mra =   [ra]
    mdec = [dec]
    ptf_sig = 2.
    ptf_dm = 0.5
    #fn11 = 'PTF_201304293062_c_p_scie_t072054_u015955703_f11_p002958_c10.ctlg'
    #fn12 = 'PTF_201304293120_c_p_scie_t072914_u015957839_f12_p002958_c10.ctlg'

    #fn1 = '//media//careyjohn/OS//Users//CareyJohn//Desktop//PSROC_klpr//K2_PTF_ha//k2_camp1//'+ID+'//'+fn11  #Linux 路徑
    #fn2 = '//media//careyjohn//OS//Users//CareyJohn//Desktop//PSROC_klpr//K2_PTF_ha//k2_camp1//'+ID+'//'+fn12
    #fn1 = 'C://Users//CareyJohn//Desktop//PSROC_klpr//K2_PTF_ha//k2_camp1//'+ID+'//'+fn11
    #fn2 = 'C://Users//CareyJohn//Desktop//PSROC_klpr//K2_PTF_ha//k2_camp1//'+ID+'//'+fn12


    fn1 = P_ha656(ha656_list[0])
    fn2 = P_ha663(ha663_list[0])

    ptf1 = pyfits.open(fn1)
    ptf2 = pyfits.open(fn2)

    ra1 = ptf1[1].data['ALPHAWIN_J2000']
    dec1 = ptf1[1].data['DELTAWIN_J2000']
    mag1 = ptf1[1].data['MAG_AUTO']+ptf1[1].data['ZEROPOINT']

    ra2 = ptf2[1].data['ALPHAWIN_J2000']
    dec2 = ptf2[1].data['DELTAWIN_J2000']
    mag2 = ptf2[1].data['MAG_AUTO']+ptf2[1].data['ZEROPOINT']
    
    def crossmatch(X1, X2, max_distance=inf, k=1):                                  
        """Cross-match the values between X1 and X2                                 

        By default, this uses a KD Tree for speed.                                  

        Parameters                                                                  
        ----------                                                                  
        X1 : array_like                                                             
            first dataset, shape(N1, D)                                             
        X2 : array_like                                                             
            second dataset, shape(N2, D)                                            
        max_distance : float (optional)                                             
            maximum radius of search.  If no point is within the given radius,      
            then inf will be returned.                                              

        Returns                                                                     
        -------                                                                     
        dist, ind: ndarrays                                                         
            The distance and index of the closest point in X2 to each point in X1   
            Both arrays are length N1.                                              
            Locations with no match are indicated by                                
            dist[i] = inf, ind[i] = N2                                              
        """                                                                         
        X1 = asarray(X1, dtype=float)                                               
        X2 = asarray(X2, dtype=float)                                               
        N1, D = X1.shape                                                            
        N2, D2 = X2.shape                                                           
        if D != D2:                                                                 
            raise ValueError('Arrays must have the same second dimension')          
        kdt = cKDTree(X2)                                                           
        dist, ind = kdt.query(X1, k=k, distance_upper_bound=max_distance)           

        return dist, ind  

    def rexmatch(radec1, radec2, max_radius, k=1):
        """call crossmatch, remove multiple matched objects in list1 to one object in list 2
        Parameters
        ----------
        radec1 : array_like
            first dataset, shape(N1, D)
        radec2 : array_like
            second dataset, shape(N2, D)
        max_radius : float 
            maximum radius of search.  If no point is within the given radius,
            then inf will be returned.

        Returns
        -------
        match1, match2: int array (they have the same size)

        dist1 : float array    
            distance between matched star
        umatch1, umatch2: int array (not have the same size)    
            The index of un-matched star in radec1 and radec2
        """
        radec1 = radec1
        radec2 = radec2

        # crossmatch catalogs
        # max_radius = 1. / 3600  # 1 arcsec
        dist, ind = crossmatch(radec1, radec2, max_radius, k=k)

        # remove multiple objects in list1 to one object in list 2
        match1 = ~isinf(dist)
        sind = sorted(ind[match1])
        us, usidx = unique(sind, return_index=True)
        mi = len(radec2)
        for i in range(1,len(us)):
            diff = usidx[i]-usidx[i-1]
            if diff > 1 :
                ri = (ind == us[i-1]).nonzero()
                dist[ri] = inf
                ind[ri] = mi

        match1 = ~isinf(dist)
        umatch1 = isinf(dist)

        inda = arange(len(radec2))
        umatch2 = list(set(inda)-set(ind))
        match2 = ind[match1]
        inda1 = arange(len(radec1))
        match1 = inda1[match1]
        umatch1 = inda1[umatch1]
        dist1 = dist[match1]

        
        return match1, match2, umatch1, umatch2, dist1
    class ptf_data:
        def __init__(self, fn1, fn2, coord, dm, sigma):
            rac, decc = coord
            if fn1.endswith('.cat'):
                with open(fn1, 'r') as fnr1:
                    ptf1 = array([i.split() for i in fnr1.readlines()[14:]]).astype(float)
                with open(fn2, 'r') as fnr2:
                    ptf2 = array([i.split() for i in fnr2.readlines()[14:]]).astype(float)
                ra1, dec1 = ptf1[:,11], ptf1[:,12]
                mag1, merr1 = ptf1[:,7], ptf1[:,8]
                x1, y1 = ptf1[:,2], ptf1[:,3]
                ra2, dec2 = ptf2[:,11], ptf2[:,12]
                mag2, merr2 = ptf2[:,7], ptf2[:,8]
                mrange = arange(7, 13, dm)
            elif fn1.endswith('.ctlg'):
                with pyfits.open(fn1) as ptf1:
                    star1 = ptf1[1].data
                    idx = (star1['FLAGS'] & 506 == 0) * (star1['IMAFLAGS_ISO'] & 1821 == 0)
                    ra1   = star1['ALPHAWIN_J2000'][idx]
                    dec1  = star1['DELTAWIN_J2000'][idx]
                    mag1  = star1['MAG_AUTO'][idx]
                    merr1 = star1['MAGERR_AUTO'][idx]
                    flux1 = star1['FLUX_AUTO'][idx]
                    fluxerr1 = star1['FLUXERR_AUTO'][idx]
                    x1, y1 = star1['XWIN_IMAGE'][idx],  star1['YWIN_IMAGE'][idx]
                    #x1, y1 = star1['XWIN_IMAGE'],  star1['YWIN_IMAGE']
                with pyfits.open(fn2) as ptf2:
                    star2 = ptf2[1].data
                    idx = (star2['FLAGS'] & 506 == 0) * (star2['IMAFLAGS_ISO'] & 1821 == 0)
                    ra2   = star2['ALPHAWIN_J2000'][idx]
                    dec2  = star2['DELTAWIN_J2000'][idx]
                    mag2  = star2['MAG_AUTO'][idx]
                    merr2 = star2['MAGERR_AUTO'][idx]
                    flux2 = star2['FLUX_AUTO'][idx]
                    fluxerr2 = star2['FLUXERR_AUTO'][idx]
                mrange = arange(ceil(mag2.min())-1, ceil(mag2.max()), dm)
                frange = arange(flux2.min(), flux2.max(), dm)


            ## match Ha656 and Ha663
            m1, m2, um1, um2, dist = rexmatch(zip(ra1,dec1), zip(ra2,dec2), 2./3600.)
            mdiff = mag2[m2]-mag1[m1]
            fdiff = flux1[m1]-flux2[m2]
            merr = sqrt(merr1[m1]**2.+merr2[m2]**2.)
            ferr = sqrt(fluxerr1[m1]**2.+fluxerr2[m2]**2.)
            ram, decm, magm = ra2[m2], dec2[m2], mag2[m2]
            xm, ym = x1[m1], y1[m1]
            self.magm, self.mdiff = magm, mdiff

            mm1, mm2, umm1, umm2, distm = rexmatch(zip(mra,mdec), zip(ra1,dec1), 2./3600.)
            #print mra, mdec, ra1, dec1, ra2, dec2
            mmm1, mmm2, ummm1, ummm2, distmm = rexmatch(zip(mra,mdec), zip(ra2,dec2), 2./3600.)
            mdiffm = mag2[mmm2]-mag1[mm2]
            fdiffm = flux1[mm2]/flux2[mmm2]
            #print mag2[mmm2], mag1[mm2], mdiffm, epic
            print mag2[mmm2] , mag1[mm2], mdiffm, flux1[mm2] , flux2[mm2] ,fdiffm, epic
            
            
            self.ras, self.decs, self.xs, self.ys, self.ms, self.mds, self.mplt = [], [], [], [], [], [], []
            for i in mrange:
                midx = (magm > i) * (magm < i+dm)
                if midx.sum():
                    mdiff_tmp, magm_tmp, merr_tmp, ram_tmp, decm_tmp, x_tmp, y_tmp = \
                        mdiff[midx], magm[midx], merr[midx], ram[midx], decm[midx], xm[midx], ym[midx]
                    mdiff_clip = sta.sigma_clip(mdiff_tmp, sigma=3, iters=5)
                    if (~mdiff_clip.mask).sum() > 0:
                        wmeandiff = average(mdiff_clip.data[~mdiff_clip.mask], weights=merr_tmp[~mdiff_clip.mask])
                        stddiff = mdiff_clip.data[~mdiff_clip.mask].std()
                        photoerr = sqrt(1./sum(1./merr_tmp[~mdiff_clip.mask]**2.))
                        errsel = sqrt(photoerr**2.+stddiff**2.)
                        self.mplt.append([i, i+dm, wmeandiff, errsel])
                        sidx = ((mdiff_tmp-merr_tmp)-wmeandiff) > (sigma*errsel)
                        if sum(sidx) !=0 :
                            self.ras += list(ram_tmp[sidx])
                            self.decs += list(decm_tmp[sidx])
                            self.xs += list(x_tmp[sidx])
                            self.ys += list(y_tmp[sidx])
                            self.ms += list(magm_tmp[sidx])
                            self.mds += list(mdiff_tmp[sidx])

                    fig = pl.figure(num=0, figsize=(24,16), dpi = 100, facecolor='w', edgecolor='k')
                    #pl.plot(magm_tmp, mdiff_tmp, '.', color='gray', markersize=4)
                    pl.plot(magm_tmp, mdiff_tmp, '.', color='gray', markersize=4)
                    pl.plot(magm_tmp[sidx], mdiff_tmp[sidx], 'ro')
                    x = (i+(i+dm))/2
                    pl.errorbar(x, wmeandiff, yerr=errsel, fmt='o', markersize=5, mec='b', mfc='b', ecolor='b')
                    pl.title("H663-H656 ="+str(mdiffm[0]))
                    pl.plot(mag2[mmm2], mdiffm, 'g*', markersize=10)
                    pl.text(mag2[mmm2]+0.01, mdiffm+0.01, ID)
                    pl.ylabel('H663-H656', size=20)
                    pl.xlabel('H663', size=20)
                    pl.xticks(size=15)
                    pl.yticks(size=15)
            def findMiddle(input_list):
                middle = float(len(input_list))/2
                if middle % 2 != 0:
                    return input_list[int(middle - .5)]
                else:
                    return (input_list[int(middle)], input_list[int(middle-1)])
            mid = findMiddle([i for i in self.ms if i <-10 and i > -11 and i is not list])
            if type(mid) is list:
                print 'standard field star', mid[0], mid[1]
            if type(mid) is not list:
                print 'standard field star', mid
            pl.show()
            #os.chdir("C:/Users/CareyJohn/Desktop/python_crawler/ktwo"+str(epic)+"_C1")
            #os.chdir("D:/python_PTF_data_k2c1/ktwo"+str(epic)+"_C1")
            #pl.show()
            #if not os.path.exists(ID+"diagram.png"):
            #    pl.savefig("D:/python_PTF_data_k2c1/ktwo"+str(epic)+"_C1/"+ID+"diagram.png" , format='png')
            #pl.clf()
            #pl.savefig("C:/Users/CareyJohn/Desktop/python_crawler/ktwo"+str(epic)+"_C1/"+ID+"diagram.png" , format='png')
            #pl.show()
            #pl.clf()
            os.chdir("..")
    ptf_data(fn1, fn2, [mra,mdec], ptf_dm, ptf_sig)

    #os.chdir("D:/kepler_ptf_research/k2_c1/python_PTF_data_k2c1/ktwo"+str(epic)+"_C1") #for windows
    #os.chdir("/media/careyjohn/DATA/python_PTF_data_k2c1/ktwo"+str(epic)+"_C1") # for Linux
    #pl.show()
    #if not os.path.exists(ID+"diagram.png"):
        #pl.show()
    #pl.savefig('//media//careyjohn/OS//Users//CareyJohn//Desktop//PSROC_klpr//K2_PTF_ha//k2_camp1//'+ID+'//diagram.png' , format='png')
    #    pl.savefig("D:/kepler_ptf_research/k2_c1/python_PTF_data_k2c1/ktwo"+str(epic)+"_C1/"+ID+"diagram.png" , format='png')
    #    pl.clf()
    #os.chdir("..")

In [8]:
#dfs['EPIC'][0:2]
ra_list = list(dfs['RA'][0:5])
dec_list = list(dfs['DEC'][0:5])
epic_list = list(dfs['EPIC'][0:5])

In [9]:
epic_list
ra_list

[174.69463500000001,
 175.25185099999999,
 175.08766799999998,
 176.619427,
 176.61401000000001]

In [16]:
for s in range(len(ra_list)):
    input_ra_dec(ra_list[s], dec_list[s], epic_list[s])
#20170309 已經能全自動存圖，目前PTF K2 C1 前五個目標可以自動存取  

[-7.26257801] [-7.46166706] [ 0.19908905] [ 965.31005859] [ 658.75042725] [ 1.20125604] 201134862
standard field star -10.3152
[-8.68230152] [-8.70197868] [ 0.01967716] [ 3025.45996094] [ 3302.81518555] [ 1.01828825] 201135941
standard field star -10.6792
[-6.89735937] [-7.22068882] [ 0.32332945] [ 773.1708374] [ 3766.18579102] [ 1.34688854] 201136343
standard field star -10.6792
[-8.19214916] [-8.24462509] [ 0.05247593] [ 1985.4095459] [ 23035.5390625] [ 1.04951906] 201137361
standard field star

IndexError: list index out of range

In [43]:
-2.5*log10(1.34688854)

-0.32332914434494642

In [50]:
os.chdir("D:\kepler_ptf_research\k2_c1\python_PTF_data_k2c1\ktwo201134862_C1")

In [51]:
ptf1 = pyfits.open('PTF_201603283150_c_p_scie_t073336_u027335665_f11_p002432_c11.ctlg')

In [52]:
ptf2 = pyfits.open('PTF_201212284122_c_p_scie_t095334_u014899117_f12_p002432_c11.ctlg')

In [60]:
ptf1[1].header

XTENSION= 'BINTABLE'           / THIS IS A BINARY TABLE (FROM THE LDACTOOLS)    
BITPIX  =                    8 /                                                
NAXIS   =                    2 /                                                
NAXIS1  =                  518 / BYTES PER ROW                                  
NAXIS2  =                 1166 / NUMBER OF ROWS                                 
PCOUNT  =                    0 / RANDOM PARAMETER COUNT                         
GCOUNT  =                    1 / GROUP COUNT                                    
TFIELDS =                   92 / FIELDS PER ROWS                                
EXTNAME = 'OBJECTS '           / TABLE NAME                                     
TTYPE1  = 'NUMBER  '           / Running object number                          
TFORM1  = '1J      '                                                            
TDISP1  = 'I10     '                                                            
TTYPE2  = 'FLAGS   '        

In [53]:
ra1 = ptf1[1].data['ALPHAWIN_J2000']

In [63]:
dec1 = ptf1[1].data['DELTAWIN_J2000']

In [64]:
ra1[1], dec1[1]

(174.79443056575226, -5.5597207098911055)

In [41]:
len(ptf1[1].data['FLUX_AUTO'])

1166

In [42]:
ptf1[1].data['ZEROPOINT']

array([ 99.98999786,  99.98999786,  99.98999786, ...,  99.98999786,
        99.98999786,  99.98999786], dtype=float32)

In [45]:
#ptf2[1].data['ZEROPOINT']

In [20]:
ptf1[1].data['MAG_AUTO']

array([-9.05555534, -7.26347303, -6.29447889, ..., -5.84507608,
       -7.55095911, -8.4882822 ], dtype=float32)

In [21]:
ptf1[1].data['FLUX_AUTO']

array([ 4190.07958984,   804.24645996,   329.45153809, ...,   217.78625488,
        1048.05383301,  2484.92211914], dtype=float32)

In [23]:
ptf1[1].data['FLUXERR_AUTO']

array([ 79.68730164,  63.51903534,  52.67891693, ...,  51.82690811,
        61.75869751,  70.85659027], dtype=float32)

In [22]:
-2.5*log10(4190.07958984)

-9.0555556809837832

In [19]:
df2 = pd.read_csv('lamostdr4_ptf_k2c1.csv')

In [22]:
len(dfs['EPIC']), len(df2['EPIC'])

(3865, 53)

In [25]:
ra = []
dec = []
epic = []
for i in range(len(df2['EPIC'])):
    for a in range(len(dfs['EPIC'])):
        if df2['EPIC'][i] == dfs['EPIC'][a]:
            print dfs['RA'][a], dfs['DEC'][a], dfs['EPIC'][a]
            ra.append(dfs['RA'][a])
            dec.append(dfs['DEC'][a])
            epic.append(dfs['EPIC'][a])

170.966085 8.467428 201951507
173.592259 2.314546 201617526
172.826447 1.082339 201536382
178.738834 7.396115 201903390
168.135601 0.498249 201498396
170.719245 3.047271 201664337
179.41823 -1.170119 201386095
170.856271 2.680977 201641211
169.196778 1.443505 201560437
171.276829 6.636349 201866368
174.648387 7.65581 201915687
168.756795 2.96378 201659277
173.854281 7.498503 201908324
174.810076 7.229846 201895565
172.029258 1.69179 201577109
174.663437 4.115649 201727980
171.301313 -0.410615 201436683
177.986745 7.524038 201909533
177.276214 7.631577 201914589
169.252691 2.655385 201639545
170.603137 5.279591 201795220
178.716903 -1.046205 201394380
179.348894 4.026309 201722715
170.674053 1.98317 201596050
171.858588 0.742116 201514262
168.696718 -0.710574 201416311
178.130523 -2.518135 201295538
171.168804 2.643205 201638723
176.718686 6.149526 201842163
177.081295 5.608945 201813040
168.328193 4.46582 201748955
170.93448 7.045302 201886530
179.714567 4.560746 201754501
170.798666 5

In [49]:
for s in range(len(ra)):
    input_ra_dec(ra[s], dec[s], epic[s])

[-9.91428185] [-10.17512798] [ 0.26084614] [ 11750.35644531] [ 87514.1953125] [ 1.27156436] 201951507
standard field star -10.6804
[-8.78907967] [-9.0094595] [ 0.22037983] [ 4015.91015625] [ 541.33190918] [ 1.22504544] 201617526
standard field star -10.1765


IndexError: list index out of range

In [44]:
if type(ra) is list:
    print 'a'

a
